In [1]:
import time
import pickle

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn import svm

import gensim.downloader as api

import torch
from torch import nn
import torch.nn.functional as F

from skorch import NeuralNetClassifier
from textrank import TextRank4Keyword

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
# nltk.download('stopwords')
# You will have to download the set of stop words the first time

# from utils import text_to_wordlist_no_stop
# from utils import text_to_wordlist_no_stop
from utils import *

if torch.cuda.is_available():
	print("WARNING: You have a CUDA device")

device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")

### Parameters

# all 3639 samples
# train 2911 samples
# text_piece = 100  # if change, also modify in chunks()
batch_size = 40
n_classes = 4

epochs = 6
log_interval = 1000
lr = 0.01
clip = 0.25


'''Model Param'''
output_size = n_classes
embedding_dim = glove_dim
hidden_dim = glove_dim
n_layers = 1

/home/yzhan273/.conda/envs/nlp/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


load word emb takes time: 100.60068774223328


In [21]:
all_files = ['severity/text_frightening_severity.csv',
             'severity/text_alcohol_severity.csv',
             'severity/text_nudity_severity.csv',
             'severity/text_violence_severity.csv',
             'severity/text_profanity_severity.csv']

# all_files = ['severity/text_alcohol_severity.csv',
#              'severity/text_nudity_severity.csv',
#              'severity/text_violence_severity.csv',
#              'severity/text_profanity_severity.csv']

f_baseline_filename = "all_file_logistic_baseline_record.txt"

with open(f_baseline_filename, "a") as f_baseline:
    for script in range(len(all_files)):
        f_baseline.write(all_files[script])
        f_baseline.write('\n')
        all_data = pd.read_csv(all_files[script], sep='\t')
        data_batch = all_data

        label_batch = data_batch['label']
        text_batch = data_batch['text']

        # reconstruct each text into long list of words
        # whole dataset text_list = num of article * one big word strings
        print('get text list')
        text_list = list(map(text_to_wordlist_no_stop, text_batch))
        # remove punctuation
        text_list = list(map(text_to_wordlist_no_punc, text_list))
        # do it again
        text_list = list(map(text_to_wordlist_no_stop, text_list))

        ##### #key word method
        # cur_time = time.time()
        # text_list = get_key_words_onlist(text_list)
        # print('get key words list takes time:', time.time() - cur_time)
        # print(text_list)

        ### word vector glove
        text_doc_emb_list = np.array(list(map(doc_emb, text_list)))

        X = text_doc_emb_list

        # label_batch_digits = label_to_one_hot(label_batch, n_classes)
        label_batch_digits = label_to_digits(label_batch)

        y = label_batch_digits

        training_size = int(0.8 * 3600) # len(X)
        validation_size = int(0.1 * 3600)
        test_size = int(0.1 * 3600)

        X_train = X[0:training_size]
        X_dev = X[training_size:(training_size + validation_size)]
        X_test = X[(training_size + validation_size):(training_size + validation_size + test_size)]

        y_train = y[0:training_size]
        y_dev = y[training_size:(training_size + validation_size)]
        y_test = y[(training_size + validation_size):(training_size + validation_size + test_size)] 

        # NN try
        X_train,X_dev,X_test  = \
        X_train.astype(np.float32),X_dev.astype(np.float32),X_test.astype(np.float32)
      

        from sklearn.metrics import f1_score    
        
        class LogisticRegression(torch.nn.Module):
            def __init__(self, input_dim, output_dim):
                super(LogisticRegression, self).__init__()
                self.linear = torch.nn.Linear(input_dim, input_dim)
                nn.init.uniform_(self.linear.weight)
#                 self.dropout = nn.Dropout(p=0.25)
                self.linear2 = torch.nn.Linear(input_dim, output_dim)
                nn.init.uniform_(self.linear2.weight)


            def forward(self, x):
                outputs = self.linear(x)
#                 outputs = self.dropout(outputs)
                outputs = self.linear2(outputs)
#                 y_pred = F.sigmoid(outputs)

                return outputs

        LR_model = LogisticRegression(embedding_dim, n_classes)

        criterion = nn.CrossEntropyLoss() # computes softmax and then the cross entropy

#         optimizer = torch.optim.SGD(LR_model.parameters(), lr=0.001)
        optimizer = torch.optim.Adam(LR_model.parameters())

        epochs = 30
        
        best_f1 = []
        for epoch in range(epochs):    
            LR_model.train()
            total_loss = 0.
            start_time = time.time()
            
            for i in range(0,X_train.shape[0],batch_size):
                
                output = LR_model(torch.Tensor(X_train[i:i + batch_size]))

                loss = criterion(output, y_train[i:i + batch_size])
    
                # loss.backward()
                loss.backward()
                optimizer.step()
#                 print(LR_model.linear.weight)
                
                total_loss += loss.item()
                
                if i % log_interval == 0 and i > 0:
                    cur_loss = total_loss / log_interval
                    elapsed = time.time() - start_time
                    print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                          'loss {:5.2f} |'.format(
                        epoch, i, X[0:partition].shape[0], lr,
                        elapsed * 1000 / log_interval, cur_loss))
                    total_loss = 0
                    start_time = time.time()
                    
            validation_output = LR_model(torch.Tensor(X_dev))
            
            
#             clas_report = classification_report(y_dev.numpy(), np.argmax(validation_output.detach().numpy(), axis=1))
#             print(clas_report)
            f1score = f1_score(y_dev.numpy(), np.argmax(validation_output.detach().numpy(), axis=1), average='macro')
            print(f1score)
            best_f1.append(f1score)
        print(all_files[script], 'best f1',max(best_f1))
        

get text list
| epoch   0 |  1000/ 2900 batches | lr 0.01 | ms/batch  0.04 | loss  0.08 |
| epoch   0 |  2000/ 2900 batches | lr 0.01 | ms/batch  0.04 | loss  0.08 |
0.195064071353025
| epoch   1 |  1000/ 2900 batches | lr 0.01 | ms/batch  0.04 | loss  0.06 |
| epoch   1 |  2000/ 2900 batches | lr 0.01 | ms/batch  0.04 | loss  0.06 |
0.14737033998723909
| epoch   2 |  1000/ 2900 batches | lr 0.01 | ms/batch  0.04 | loss  0.06 |
| epoch   2 |  2000/ 2900 batches | lr 0.01 | ms/batch  0.03 | loss  0.06 |
0.15790655423008365
| epoch   3 |  1000/ 2900 batches | lr 0.01 | ms/batch  0.04 | loss  0.05 |
| epoch   3 |  2000/ 2900 batches | lr 0.01 | ms/batch  0.04 | loss  0.04 |
0.301935233950663
| epoch   4 |  1000/ 2900 batches | lr 0.01 | ms/batch  0.04 | loss  0.04 |
| epoch   4 |  2000/ 2900 batches | lr 0.01 | ms/batch  0.03 | loss  0.04 |
0.2652354394115312
| epoch   5 |  1000/ 2900 batches | lr 0.01 | ms/batch  0.04 | loss  0.05 |
| epoch   5 |  2000/ 2900 batches | lr 0.01 | ms/batch 

KeyboardInterrupt: 